In [1]:
import pdfExtractor
from pdfExtractor import PaperPageInfo, TextBox, TextLine, Text

In [2]:
path = pdfExtractor.testpath(15)
print path

./test-samples/Articles/1302.4102v1 copy.pdf


In [3]:
first_page_xml = pdfExtractor.extractPagesXml(path)[0]
page_obj = pdfExtractor.getXmlObject(first_page_xml)

In [4]:
page = PaperPageInfo(pdfExtractor.getPages(page_obj)[0])
print('page size:')
print(page.getBoxCenter())

page size:
[ 306.  396.]


In [5]:
# print len(page.getTextBoxes())

In [6]:
textboxes = page.getTextBoxes()
# for x in range(len(textboxes)):
#     print('textbox[' + str(x) + '].boxcenter: ')
#     print(textboxes[x].getBoxCenter())

In [10]:
import math
DEBUG = False
titlecandidates = []
tol = 50
pagecenterX = page.getBoxCenter()[0]
for textbox in textboxes:
    if len(textbox.getTextLines()) > 3:
        continue
    if DEBUG: print('id=' + textbox.getId())
    needsort = False
    if abs(textbox.getBoxCenter()[0] - pagecenterX) < tol or len(titlecandidates) == 0:
        titlecandidates.append(textbox)
        needsort = True
    else:
        if abs(textbox.getBoxCenter()[0] - pagecenterX) < abs(titlecandidates[0].getBoxCenter()[0] - pagecenterX):
            titlecandidates[0] = textbox
            needsort = True
        
    if len(titlecandidates) > 1 and needsort:
        firstcandidate = titlecandidates[0]
        for x in range(len(titlecandidates)):
            if x == 0:
                continue
            if abs(titlecandidates[x].getBoxCenter()[0] - pagecenterX) > abs(firstcandidate.getBoxCenter()[0] - pagecenterX):
                titlecandidates[0] = titlecandidates[x]
                titlecandidates[x] = firstcandidate
                firstcandidate = titlecandidates[0]
        if DEBUG:
            for x in range(len(titlecandidates)):
                print('title[' + str(x) + ', id=' + titlecandidates[x].getId() + '].boxcenter: ')
                print(titlecandidates[x].getBoxCenter())
            print('------------------')

In [11]:
for x in range(len(titlecandidates)-1):
    for y in range(len(titlecandidates)-x-1):
        xcand = titlecandidates[x]
        ycand = titlecandidates[y+x+1]
        if abs(xcand.getBoxCenter()[0] - pagecenterX) > abs(ycand.getBoxCenter()[0] - pagecenterX):
            titlecandidates[x] = ycand
            titlecandidates[y+x+1] = xcand
            if not DEBUG: continue
            print('swap(' + str(x) + ' and ' + str(y+x+1) + ')')
            for idx in range(len(titlecandidates)):
                print('title[' + str(idx) + ', id=' + titlecandidates[idx].getId() + '].boxcenter: ')
                print(titlecandidates[idx].getBoxCenter())
            print('---------------------')
        
for x in range(len(titlecandidates)):
    print('title[' + str(x) + ', id=' + titlecandidates[x].getId() + '].boxcenter: ')
    print(titlecandidates[x].getBoxCenter())

title[0, id=1].boxcenter: 
[ 307.7985  704.885 ]
title[1, id=0].boxcenter: 
[ 308.047  735.062]
title[2, id=2].boxcenter: 
[ 308.048  685.379]
title[3, id=4].boxcenter: 
[ 308.048  664.129]
title[4, id=3].boxcenter: 
[ 308.049  674.563]


In [12]:
print('pagecenter:')
print(page.getBoxCenter())
for candidate in titlecandidates:
    c = candidate.getBoxCenter()
    print('candidate[id=' + candidate.getId() + ', boxcenter=(' + str(c[0]) + ', ' + str(c[1]) + ']: ')
    print(str(len(candidate.getTextLines())) + ' lines')
    for textline in candidate.getTextLines():
        print('textSize: ' + textline.getTexts()[0].getTextSize())
        print('font: ' + textline.getTexts()[0].getTextFont())
        print(textline.getLineText())
    print('----------------------------------')

pagecenter:
[ 306.  396.]
candidate[id=1, boxcenter=(307.7985, 704.885]: 
2 lines
textSize: 12.533
font: IXIKVN+SFRM1000
M. Brust,1 C. Schaefer,1 R. Doerr,1 L. Pan,2 M. Garcia,2 P. E. Arratia,2 and C. Wagner1, 

textSize: 7.556
font: NSLYGY+SFRM0600
1Experimentalphysik, Universitt des Saarlandes, Postfach 151150, 66041 Saarbrcken, Germany

----------------------------------
candidate[id=0, boxcenter=(308.047, 735.062]: 
1 lines
textSize: 14.836
font: UBDIWP+SFBX1200
Rheology of human blood plasma: Viscoelastic versus Newtonian behavior

----------------------------------
candidate[id=2, boxcenter=(308.048, 685.379]: 
1 lines
textSize: 7.556
font: NSLYGY+SFRM0600
2Department of Mechanical Engineering & Applied Mechanics,

----------------------------------
candidate[id=4, boxcenter=(308.048, 664.129]: 
1 lines
textSize: 11.289
font: WGJANA+SFRM0900
(Dated: February 19, 2013)

----------------------------------
candidate[id=3, boxcenter=(308.049, 674.563]: 
1 lines
textSize: 11.235
font: